In [1]:
# load binary file
import numpy as np
import time
import suite2p, register, dcnv, celldetect

def tic():
    return time.time()
def toc(i0):
    return time.time() - i0

ops = {
    'diameter': 14,
    'tau': 2.,
    'fs': 2.5,
    'data_path': 'H:/DATA/2017-10-13/',
    'subfolders': ('1'),
    'numworkers':-1
      }

# get defaults options
ops0 = suite2p.default_ops()

# combine with user options
ops = {**ops0, **ops} 

In [2]:
i0 = tic()
# copy tiff to a binary
ops1 = register.tiff_to_binary(ops)
# register tiff
ops1 = register.register_binary(ops1)

print(toc(i0))

151.96024656295776


In [ ]:
import imp
imp.reload(celldetect)
U,sdmov      = celldetect.getSVDdata(ops1[0])

In [ ]:
ops = ops1[0]
S, StU , StS = celldetect.getStU(ops, U)
# get ROIs
ops, stat, cell_masks, neuropil_masks, mPix, mLam = celldetect.sourcery(ops, U, S, StU, StS)
# extract fluorescence and neuropil
F, Fneu = celldetect.extractF(ops, stat, cell_masks, neuropil_masks, mPix, mLam)
# deconvolve fluorescence
spks = dcnv.oasis(F - ops['neucoeff'] * Fneu, ops)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
%matplotlib inline
import scipy.stats as stats

dF = F - 0.7*Fneu
dF = stats.zscore(dF, axis=1)

plt.figure(figsize=(18,18))
plt.imshow(dF[:200,:], vmax=5)
plt.show()